In [1]:
import pandas as pd
import sqlite3

# Load Excel file
df = pd.read_excel("../data/cleaned_retail.xlsx", engine="openpyxl")

# Create SQLite database and table
conn = sqlite3.connect("retail.db")
df.to_sql("transactions", conn, if_exists="replace", index=False)

# Create temp table for non-cancelled transactions
conn.execute("""
    CREATE TEMP TABLE clean_transactions AS
    SELECT *
    FROM transactions
    WHERE CancellationFlag IS NULL OR TRIM(CancellationFlag) = '';
""")

# Query 1: Total number of valid transactions
query1 = """
    SELECT COUNT(DISTINCT InvoiceNo) AS valid_transaction_count
    FROM clean_transactions;
"""
print("\n Total Valid Transactions:")
print(pd.read_sql_query(query1, conn))

# Query 2: Top 10 sales by country
query2 = """
    SELECT 
      Country,
      printf('%,d', COUNT(DISTINCT InvoiceNo)) AS Total_transactions,
      printf('£%,.2f', SUM(Quantity * UnitPrice)) AS Formatted_revenue
    FROM clean_transactions
    GROUP BY Country
    ORDER BY SUM(Quantity * UnitPrice) DESC
    LIMIT 10;
"""
print("\n Top 10 Sales by Country:")
print(pd.read_sql_query(query2, conn))

# Query 3: Top 10 selling products
query3 = """
    SELECT 
      StockCode, 
      Description, 
      printf('%,d', SUM(Quantity)) AS total_sold
    FROM clean_transactions
    GROUP BY StockCode, Description
    ORDER BY SUM(Quantity) DESC
    LIMIT 10;
"""
print("\n Top 10 Selling Products:")
print(pd.read_sql_query(query3, conn))

# Close connection
conn.close()


 Total Valid Transactions:
   valid_transaction_count
0                    19745

 Top 10 Sales by Country:
          Country Total_transactions Formatted_revenue
0  United Kingdom             17,863     £8,693,481.34
1     Netherlands                 94       £285,446.34
2            EIRE                281       £276,150.18
3         Germany                451       £227,569.65
4          France                386       £207,743.45
5       Australia                 54       £138,219.71
6           Spain                 89        £61,530.36
7     Switzerland                 52        £56,905.95
8         Belgium                 98        £41,196.34
9           Japan                 19        £37,416.37

 Top 10 Selling Products:
  StockCode                         Description total_sold
0     84077   WORLD WAR 2 GLIDERS ASSTD DESIGNS     54,903
1    85099B             JUMBO BAG RED RETROSPOT     47,823
2    85123A  WHITE HANGING HEART T-LIGHT HOLDER     37,021
3     22197            